# Final Model

## Model 5: Transfer learning model with denoised images and augmentation, 16 epochs

This model utilizes transfer learning and the functional Keras API to produce a mixed convolutional model on denoised images. Because of imbalance, the melanoma images are augmented. A dropout layer and L2 regularization were included to prevent overfitting. The model utilizes the [Xception](https://keras.io/api/applications/xception/) architecture, trained on the famous [ImageNet](https://image-net.org/) image database.

In [1]:
import tensorflow as tf
from tensorflow.keras import Input, Model
from tensorflow.keras.applications import Xception
from tensorflow.keras.layers import Concatenate, Conv2D, Dense, Dropout, GlobalAveragePooling2D
from tensorflow.keras.metrics import AUC, Recall
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.regularizers import l2
from sklearn.metrics import classification_report
import pickle
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession
import time

config = ConfigProto()
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)

In [2]:
meta = pd.read_csv('../processed_data/train.csv', index_col=0)
meta.head()

,age_approx,Unknown,female,male,head/neck,lower extremity,oral/genital,palms/soles,torso,upper extremity,target
0,45.0,0,0,1,1,0,0,0,0,0,0
1,45.0,0,1,0,0,0,0,0,0,1,0
2,50.0,0,1,0,0,1,0,0,0,0,0
3,45.0,0,1,0,1,0,0,0,0,0,0
4,55.0,0,1,0,0,0,0,0,0,1,0


### Metadata

In [3]:
y = meta['target'].values.reshape(-1,1)
X = meta.drop('target', axis=1)
meta_train, meta_test, y_train, y_test = train_test_split(X, y, test_size=0.3 ,random_state=867_5309, stratify=y)

In [4]:
meta_train = meta_train.astype('ushort')
meta_test = meta_test.astype('ushort')

In [5]:
meta_train.shape

(23188, 10)

In [6]:
y_train.shape

(23188, 1)

### Image data

In [7]:
PATH = '../processed_data/processed_train_img_array.pkl'
with open(PATH, 'rb') as file:
    img_array = pickle.load(file)
img_array = img_array.astype('ushort')
img_train, img_test, _1, _2 = train_test_split(img_array, y, test_size=0.3, stratify=y, random_state=867_5309) # Should split exactly the same as above

In [8]:
img_train.shape

(23188, 80, 120, 3)

In [9]:
img_test.shape

(9938, 80, 120, 3)

### Augmentation

In [10]:
# Prepare augmentation files
augmentation_files = pd.read_csv('../data/train.csv', usecols=['image_name'])
augmentation_files = pd.concat([augmentation_files,meta], axis=1)
augmentation_files = augmentation_files[augmentation_files['target'] == 1]
augmentation_files['index'] = range(len(augmentation_files))

raw_files_path = '../data/jpeg/train/'
augmentation_files['image_name'] = augmentation_files['image_name'].map(lambda x: raw_files_path+x+'.jpg')

In [11]:
# Instantiate data generator
aug = ImageDataGenerator(rotation_range=270,
                        height_shift_range=0.1,
                        width_shift_range=0.1,
                        brightness_range=[0.3,0.9],
                        channel_shift_range=50.0,
                        horizontal_flip=True,
                        vertical_flip=True,
                        shear_range=25.0)

In [12]:
ITERATIONS = img_train.shape[0]//len(augmentation_files)
aug.fit(img_train)
img_generator = aug.flow_from_dataframe(dataframe=augmentation_files,
                                        x_col='image_name',
                                        y_col='index',
                                        class_mode='raw',
                                        shuffle=False,
                                        target_size=img_train.shape[1:3],
                                        seed=867_5309
                                       )

# Augment data additional 
img_y = np.empty((1,1))
iteration = 0
for x_batch in img_generator:
    iteration += 1
    img_train = np.concatenate((img_train, x_batch[0]), axis=0)
    img_y = np.concatenate((img_y, x_batch[1].reshape(-1,1)), axis=0)
    print(f'Batch {iteration} completed.')
    if iteration == ITERATIONS:
        break
img_y = img_y[1:]

Found 584 validated image filenames.
Batch 1 completed.
Batch 2 completed.
Batch 3 completed.
Batch 4 completed.
Batch 5 completed.
Batch 6 completed.
Batch 7 completed.
Batch 8 completed.
Batch 9 completed.
Batch 10 completed.
Batch 11 completed.
Batch 12 completed.
Batch 13 completed.
Batch 14 completed.
Batch 15 completed.
Batch 16 completed.
Batch 17 completed.
Batch 18 completed.
Batch 19 completed.
Batch 20 completed.
Batch 21 completed.
Batch 22 completed.
Batch 23 completed.
Batch 24 completed.
Batch 25 completed.
Batch 26 completed.
Batch 27 completed.
Batch 28 completed.
Batch 29 completed.
Batch 30 completed.
Batch 31 completed.
Batch 32 completed.
Batch 33 completed.
Batch 34 completed.
Batch 35 completed.
Batch 36 completed.
Batch 37 completed.
Batch 38 completed.
Batch 39 completed.


In [13]:
# Join metadata
new_metadata = pd.DataFrame(img_y, columns=['index']).astype('int')
new_metadata = pd.merge(new_metadata,augmentation_files,
                        on='index',
                        how='left',
                        validate='many_to_many')[meta.columns]

meta_train = pd.concat([meta_train, new_metadata.drop('target', axis=1)])
y_train = np.concatenate([y_train, new_metadata['target'].values.reshape(-1,1)])

### Model

In [14]:
LEARNING_RATE = 0.001
recall = Recall(name='recall')

# Define inputs
meta_inputs = Input(shape=(10,))
img_inputs = Input(shape=(80,120,3,))

# Model 1
meta_model = Dense(4, activation='relu')(meta_inputs)
meta_model = Dense(4, activation='relu')(meta_model)

# Model 2
img_model = Xception(include_top=False, input_shape=(80,120,3,))(img_inputs)
img_model = Conv2D(8, kernel_size=(5,5),
                         padding='same',
                         activation='relu')(img_model)
img_model = GlobalAveragePooling2D()(img_model)
img_model = Dropout(rate=0.35)(img_model)

# Merge models
merged_model = Concatenate()([meta_model, img_model])
merged_model = Dense(4, activation='relu',
                     kernel_regularizer=l2(l=0.04))(merged_model)
merged_output = Dense(1, activation='sigmoid')(merged_model)


# Define functional model
model = Model(inputs=[meta_inputs, img_inputs], outputs=merged_output)

# Compile model
auc = AUC(name = 'auc')
model.compile(Adam(learning_rate=LEARNING_RATE), loss='binary_crossentropy', metrics=[auc, recall])
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 80, 120, 3)] 0                                            
__________________________________________________________________________________________________
xception (Functional)           (None, 3, 4, 2048)   20861480    input_2[0][0]                    
__________________________________________________________________________________________________
input_1 (InputLayer)            [(None, 10)]         0                                            
__________________________________________________________________________________________________
conv2d_4 (Conv2D)               (None, 3, 4, 8)      409608      xception[0][0]                   
______________________________________________________________________________________________

See `fchollet`'s discussion of class weights [here](https://keras.io/examples/structured_data/imbalanced_classification/).

In [15]:
weight_for_0 = 1.0 / ((len(y_train)-y_train.sum())/len(y_train))
weight_for_1 = 1.0 / (y_train.sum()/len(y_train))
class_weight = {0: weight_for_0, 1: weight_for_1}
class_weight

{0: 1.070635234206945, 1: 15.157240522063391}

In [16]:
EPOCHS = 16
BATCH_SIZE = 10

In [ ]:
history = model.fit(x = [meta_train, img_train], y = y_train,
                    class_weight=class_weight,
                    epochs = EPOCHS,
                    batch_size = BATCH_SIZE,
                    validation_data=([meta_test, img_test], y_test))

Epoch 1/16
2439/2439 [==============================] - 157s 60ms/step - loss: 2.3069 - auc: 0.6551 - recall: 0.6963 - val_loss: 0.7756 - val_auc: 0.7758 - val_recall: 0.8114
Epoch 2/16
2439/2439 [==============================] - 146s 60ms/step - loss: 1.2472 - auc: 0.8259 - recall: 0.7944 - val_loss: 0.4941 - val_auc: 0.8043 - val_recall: 0.6857
Epoch 3/16
1096/2439 [============>.................] - ETA: 1:10 - loss: 1.0218 - auc: 0.8660 - recall: 0.8116

In [ ]:
X = np.arange(0, EPOCHS)

plt.figure(1, figsize = (20, 12))
plt.subplot(1,2,1)
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.plot(X, history.history['loss'], label = 'Training Loss')
plt.plot(X, history.history['val_loss'], label = 'Validation Loss')
plt.grid(True)
plt.xticks(X+1)
plt.legend()
plt.title('Loss', fontsize=30)

plt.subplot(1,2,2)
plt.xlabel('Epochs')
plt.ylabel('AUC')
plt.plot(X, history.history['auc'], label = 'Training AUC')
plt.plot(X, history.history['val_auc'], label = 'Validation AUC')
plt.grid(True)
plt.xticks(X+1)
plt.legend()
_ = plt.title('AUC', fontsize=30)

In [ ]:
y_pred = model.predict([meta_test, img_test])
y_pred = np.array([int(i) for i in y_pred > 0.5])
print(classification_report(y_test, y_pred))